# Module 2.2: Moving Averages and Trends

## Overview
This notebook covers moving averages and trend detection techniques. You'll learn to implement Simple Moving Average (SMA), Exponential Moving Average (EMA), MACD, and develop trend-following strategies.

## Learning Objectives
- Understand different types of moving averages
- Implement SMA and EMA from scratch
- Create MACD indicator and signals
- Detect trends using moving averages
- Build trend-following trading strategies
- Backtest moving average strategies

## Contents
1. Introduction to Moving Averages
2. Simple Moving Average (SMA)
3. Exponential Moving Average (EMA)
4. MACD (Moving Average Convergence Divergence)
5. Trend Detection with Moving Averages
6. Trading Strategies
7. Backtesting and Performance Analysis

## 1. Setup and Data Import

First, let's import the necessary libraries and load some sample data.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import warnings
import requests
import os
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Download sample data - Apple stock (AAPL)
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-12-31'
session = requests.Session()
proxy_host = "localhost"
proxy_port = 10809
proxy_url = f"http://{proxy_host}:{proxy_port}"
session.proxies.update(
    {
        "http": proxy_url,
        "https": proxy_url,
    }
)
os.environ['HTTP_PROXY'] = proxy_url
os.environ['HTTPS_PROXY'] = proxy_url
ticker = yf.Ticker(symbol)
ticker.session = session
# Download data
data = ticker.history(start=start_date, end=end_date, interval='1d')
data = data.round(2)

print(f"Downloaded {len(data)} days of data for {symbol}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print("\nFirst 5 rows:")
data.head()

Downloaded 1257 days of data for AAPL
Date range: 2020-01-02 to 2024-12-30

First 5 rows:


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02 00:00:00-05:00,71.63,72.68,71.37,72.62,135480400,0.0,0.0
2020-01-03 00:00:00-05:00,71.85,72.68,71.69,71.91,146322800,0.0,0.0
2020-01-06 00:00:00-05:00,71.03,72.53,70.78,72.49,118387200,0.0,0.0
2020-01-07 00:00:00-05:00,72.50,72.75,71.93,72.15,108872000,0.0,0.0
2020-01-08 00:00:00-05:00,71.85,73.61,71.85,73.31,132079200,0.0,0.0


## 2. Simple Moving Average (SMA)

The Simple Moving Average is the arithmetic mean of prices over a specified period. It's the most basic form of moving average.

### Formula:
**SMA = (P₁ + P₂ + ... + Pₙ) / n**

Where:
- P₁, P₂, ..., Pₙ are the prices
- n is the number of periods

In [3]:
def simple_moving_average(prices, window):
    """
    Calculate Simple Moving Average
    
    Parameters:
    prices (pd.Series): Price series
    window (int): Number of periods for moving average
    
    Returns:
    pd.Series: Simple moving average
    """
    return prices.rolling(window=window).mean()

# Calculate different SMA periods
data['SMA_10'] = simple_moving_average(data['Close'], 10)
data['SMA_20'] = simple_moving_average(data['Close'], 20)
data['SMA_50'] = simple_moving_average(data['Close'], 50)
data['SMA_200'] = simple_moving_average(data['Close'], 200)

print("SMA calculations completed")
print("\nSample data with SMAs:")
data[['Close', 'SMA_10', 'SMA_20', 'SMA_50', 'SMA_200']].tail()

SMA calculations completed

Sample data with SMAs:


,Close,SMA_10,SMA_20,SMA_50,SMA_200
Date,,,,,
2024-12-23 00:00:00-05:00,254.66,249.645,244.4380,234.7138,210.47690
2024-12-24 00:00:00-05:00,257.58,250.686,245.7015,235.2556,210.90625
2024-12-26 00:00:00-05:00,258.40,251.936,246.8970,235.7630,211.33730
2024-12-27 00:00:00-05:00,254.97,252.697,247.9275,236.2430,211.76165
2024-12-30 00:00:00-05:00,251.59,253.103,248.6690,236.6480,212.15980


In [4]:
# Visualize SMAs
fig = go.Figure()

# Add price and SMAs
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], 
                        name='Close Price', line=dict(color='black', width=1)))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_10'], 
                        name='SMA 10', line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_20'], 
                        name='SMA 20', line=dict(color='orange', width=1)))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'], 
                        name='SMA 50', line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_200'], 
                        name='SMA 200', line=dict(color='red', width=2)))

fig.update_layout(
    title=f'{symbol} Price with Simple Moving Averages',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    height=600,
    showlegend=True
)

fig.show()

## 3. Exponential Moving Average (EMA)

The Exponential Moving Average gives more weight to recent prices, making it more responsive to new information than SMA.

### Formula:
**EMA_today = (Price_today × α) + (EMA_yesterday × (1 - α))**

Where:
- α = 2 / (n + 1) (smoothing factor)
- n = number of periods

In [5]:
def exponential_moving_average(prices, window):
    """
    Calculate Exponential Moving Average
    
    Parameters:
    prices (pd.Series): Price series
    window (int): Number of periods for moving average
    
    Returns:
    pd.Series: Exponential moving average
    """
    return prices.ewm(span=window, adjust=False).mean()

# Alternative manual implementation
def ema_manual(prices, window):
    """
    Manual implementation of EMA for educational purposes
    """
    alpha = 2 / (window + 1)
    ema = np.zeros(len(prices))
    ema[0] = prices.iloc[0]  # First value is the first price
    
    for i in range(1, len(prices)):
        ema[i] = alpha * prices.iloc[i] + (1 - alpha) * ema[i-1]
    
    return pd.Series(ema, index=prices.index)

# Calculate EMAs
data['EMA_10'] = exponential_moving_average(data['Close'], 10)
data['EMA_20'] = exponential_moving_average(data['Close'], 20)
data['EMA_50'] = exponential_moving_average(data['Close'], 50)
data['EMA_200'] = exponential_moving_average(data['Close'], 200)

print("EMA calculations completed")
print("\nSample data with EMAs:")
data[['Close', 'EMA_10', 'EMA_20', 'EMA_50', 'EMA_200']].tail()

EMA calculations completed

Sample data with EMAs:


,Close,EMA_10,EMA_20,EMA_50,EMA_200
Date,,,,,
2024-12-23 00:00:00-05:00,254.66,249.440416,244.864807,236.922090,216.279912
2024-12-24 00:00:00-05:00,257.58,250.920341,246.075778,237.732204,216.690858
2024-12-26 00:00:00-05:00,258.40,252.280279,247.249513,238.542706,217.105874
2024-12-27 00:00:00-05:00,254.97,252.769319,247.984798,239.186913,217.482632
2024-12-30 00:00:00-05:00,251.59,252.554897,248.328150,239.673309,217.822009


In [6]:
# Compare SMA vs EMA
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=['SMA vs EMA Comparison', 'Difference (EMA - SMA)'],
                    vertical_spacing=0.1)

# Top subplot: Price with SMA and EMA
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], 
                        name='Close Price', line=dict(color='black')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_20'], 
                        name='SMA 20', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_20'], 
                        name='EMA 20', line=dict(color='red')), row=1, col=1)

# Bottom subplot: Difference
difference = data['EMA_20'] - data['SMA_20']
fig.add_trace(go.Scatter(x=data.index, y=difference, 
                        name='EMA - SMA', line=dict(color='green')), row=2, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_layout(height=800, title_text="SMA vs EMA Analysis")
fig.show()

# Print statistics
print(f"Average difference (EMA - SMA): ${difference.mean():.2f}")
print(f"Standard deviation of difference: ${difference.std():.2f}")
print(f"Maximum positive difference: ${difference.max():.2f}")
print(f"Maximum negative difference: ${difference.min():.2f}")

Average difference (EMA - SMA): $0.02
Standard deviation of difference: $1.43
Maximum positive difference: $3.70
Maximum negative difference: $-4.40


## 4. MACD (Moving Average Convergence Divergence)

MACD is a trend-following momentum indicator that shows the relationship between two moving averages.

### Components:
1. **MACD Line**: 12-period EMA - 26-period EMA
2. **Signal Line**: 9-period EMA of MACD Line
3. **Histogram**: MACD Line - Signal Line

In [7]:
def calculate_macd(prices, fast_period=12, slow_period=26, signal_period=9):
    """
    Calculate MACD indicator
    
    Parameters:
    prices (pd.Series): Price series
    fast_period (int): Fast EMA period (default 12)
    slow_period (int): Slow EMA period (default 26)
    signal_period (int): Signal line EMA period (default 9)
    
    Returns:
    tuple: (macd_line, signal_line, histogram)
    """
    # Calculate EMAs
    ema_fast = exponential_moving_average(prices, fast_period)
    ema_slow = exponential_moving_average(prices, slow_period)
    
    # Calculate MACD line
    macd_line = ema_fast - ema_slow
    
    # Calculate signal line
    signal_line = exponential_moving_average(macd_line, signal_period)
    
    # Calculate histogram
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

# Calculate MACD
data['MACD'], data['MACD_Signal'], data['MACD_Histogram'] = calculate_macd(data['Close'])

print("MACD calculations completed")
print("\nSample MACD data:")
data[['Close', 'MACD', 'MACD_Signal', 'MACD_Histogram']].tail()

MACD calculations completed

Sample MACD data:


,Close,MACD,MACD_Signal,MACD_Histogram
Date,,,,
2024-12-23 00:00:00-05:00,254.66,5.758346,5.214265,0.544082
2024-12-24 00:00:00-05:00,257.58,6.059792,5.383370,0.676422
2024-12-26 00:00:00-05:00,258.40,6.292323,5.565161,0.727162
2024-12-27 00:00:00-05:00,254.97,6.129179,5.677964,0.451215
2024-12-30 00:00:00-05:00,251.59,5.661883,5.674748,-0.012865


In [8]:
# Visualize MACD
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=[f'{symbol} Price', 'MACD Indicator'],
                    vertical_spacing=0.1,
                    row_heights=[0.7, 0.3])

# Top subplot: Price
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], 
                        name='Close Price', line=dict(color='black')), row=1, col=1)

# Bottom subplot: MACD
fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], 
                        name='MACD', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['MACD_Signal'], 
                        name='Signal', line=dict(color='red')), row=2, col=1)

# Add histogram
colors = ['green' if x >= 0 else 'red' for x in data['MACD_Histogram']]
fig.add_trace(go.Bar(x=data.index, y=data['MACD_Histogram'], 
                    name='Histogram', marker_color=colors, opacity=0.7), row=2, col=1)

fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_layout(height=800, title_text="Price and MACD Analysis")
fig.show()

## 5. MACD Trading Signals

Common MACD trading signals:
1. **Signal Line Crossover**: Buy when MACD crosses above signal line, sell when below
2. **Zero Line Crossover**: Buy when MACD crosses above zero, sell when below
3. **Divergence**: When price and MACD move in opposite directions

In [9]:
def generate_macd_signals(data):
    """
    Generate MACD trading signals
    
    Parameters:
    data (pd.DataFrame): DataFrame with MACD columns
    
    Returns:
    pd.DataFrame: DataFrame with signal columns added
    """
    df = data.copy()
    
    # Signal line crossover signals
    df['MACD_Signal_Cross'] = 0
    df.loc[df['MACD'] > df['MACD_Signal'], 'MACD_Signal_Cross'] = 1  # Bullish
    df.loc[df['MACD'] < df['MACD_Signal'], 'MACD_Signal_Cross'] = -1  # Bearish
    
    # Zero line crossover signals
    df['MACD_Zero_Cross'] = 0
    df.loc[df['MACD'] > 0, 'MACD_Zero_Cross'] = 1  # Bullish
    df.loc[df['MACD'] < 0, 'MACD_Zero_Cross'] = -1  # Bearish
    
    # Detect signal changes (entry/exit points)
    df['Signal_Cross_Change'] = df['MACD_Signal_Cross'].diff()
    df['Zero_Cross_Change'] = df['MACD_Zero_Cross'].diff()
    
    # Mark entry points
    df['Buy_Signal'] = (df['Signal_Cross_Change'] == 2).astype(int)  # -1 to 1
    df['Sell_Signal'] = (df['Signal_Cross_Change'] == -2).astype(int)  # 1 to -1
    
    return df

# Generate signals
data = generate_macd_signals(data)

# Find signal points
buy_signals = data[data['Buy_Signal'] == 1]
sell_signals = data[data['Sell_Signal'] == 1]

print(f"Total buy signals: {len(buy_signals)}")
print(f"Total sell signals: {len(sell_signals)}")
print("\nFirst 5 buy signals:")
print(buy_signals[['Close', 'MACD', 'MACD_Signal']].head())

Total buy signals: 48
Total sell signals: 48

First 5 buy signals:
                           Close      MACD  MACD_Signal
Date                                                   
2020-01-08 00:00:00-05:00  73.31 -0.004394    -0.026182
2020-02-06 00:00:00-05:00  78.63  0.962207     0.917933
2020-02-12 00:00:00-05:00  79.30  1.029829     0.950881
2020-03-26 00:00:00-04:00  62.64 -3.748342    -3.762547
2020-06-08 00:00:00-04:00  81.04  2.363217     2.331333


In [10]:
# Visualize signals
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=[f'{symbol} Price with MACD Signals', 'MACD Indicator'],
                    vertical_spacing=0.1,
                    row_heights=[0.7, 0.3])

# Top subplot: Price with signals
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], 
                        name='Close Price', line=dict(color='black')), row=1, col=1)

# Add buy signals
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], 
                        mode='markers', name='Buy Signal', 
                        marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)

# Add sell signals
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], 
                        mode='markers', name='Sell Signal', 
                        marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)

# Bottom subplot: MACD
fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], 
                        name='MACD', line=dict(color='blue')), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['MACD_Signal'], 
                        name='Signal', line=dict(color='red')), row=2, col=1)

fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_layout(height=800, title_text="MACD Trading Signals")
fig.show()

## 6. Trend Detection with Moving Averages

Moving averages can be used to identify trends:
- **Uptrend**: Price above MA, MA sloping up
- **Downtrend**: Price below MA, MA sloping down
- **Sideways**: Price oscillating around MA

In [11]:
def detect_trend(data, ma_column='SMA_50', lookback=5):
    """
    Detect trend using moving average
    
    Parameters:
    data (pd.DataFrame): DataFrame with price and MA data
    ma_column (str): Moving average column to use
    lookback (int): Periods to look back for trend slope
    
    Returns:
    pd.DataFrame: DataFrame with trend columns added
    """
    df = data.copy()
    
    # Calculate MA slope
    df['MA_Slope'] = df[ma_column].rolling(window=lookback).apply(
        lambda x: (x.iloc[-1] - x.iloc[0]) / lookback if len(x) == lookback else 0
    )
    
    # Price relative to MA
    df['Price_vs_MA'] = df['Close'] - df[ma_column]
    
    # Trend classification
    conditions = [
        (df['Price_vs_MA'] > 0) & (df['MA_Slope'] > 0),  # Strong uptrend
        (df['Price_vs_MA'] > 0) & (df['MA_Slope'] <= 0),  # Weak uptrend
        (df['Price_vs_MA'] <= 0) & (df['MA_Slope'] > 0),  # Weak downtrend
        (df['Price_vs_MA'] <= 0) & (df['MA_Slope'] <= 0)  # Strong downtrend
    ]
    
    choices = ['Strong Up', 'Weak Up', 'Weak Down', 'Strong Down']
    df['Trend'] = np.select(conditions, choices, default='Sideways')
    
    # Numerical trend (for easier analysis)
    trend_map = {'Strong Up': 2, 'Weak Up': 1, 'Sideways': 0, 'Weak Down': -1, 'Strong Down': -2}
    df['Trend_Numeric'] = df['Trend'].map(trend_map)
    
    return df

# Detect trends
data = detect_trend(data)

# Analyze trend distribution
trend_counts = data['Trend'].value_counts()
print("Trend Distribution:")
for trend, count in trend_counts.items():
    percentage = (count / len(data)) * 100
    print(f"{trend}: {count} days ({percentage:.1f}%)")

print("\nSample trend data:")
data[['Close', 'SMA_50', 'MA_Slope', 'Price_vs_MA', 'Trend']].tail(10)

Trend Distribution:
Strong Up: 647 days (51.5%)
Strong Down: 320 days (25.5%)
Weak Down: 119 days (9.5%)
Weak Up: 118 days (9.4%)
Sideways: 53 days (4.2%)

Sample trend data:


,Close,SMA_50,MA_Slope,Price_vs_MA,Trend
Date,,,,,
2024-12-16 00:00:00-05:00,250.44,232.1452,0.35572,18.2948,Strong Up
2024-12-17 00:00:00-05:00,252.87,232.7844,0.40164,20.0856,Strong Up
2024-12-18 00:00:00-05:00,247.45,233.2338,0.40604,14.2162,Strong Up
2024-12-19 00:00:00-05:00,249.19,233.6428,0.39724,15.5472,Strong Up
2024-12-20 00:00:00-05:00,253.88,234.1556,0.40208,19.7244,Strong Up
2024-12-23 00:00:00-05:00,254.66,234.7138,0.38588,19.9462,Strong Up
2024-12-24 00:00:00-05:00,257.58,235.2556,0.40436,22.3244,Strong Up
2024-12-26 00:00:00-05:00,258.40,235.7630,0.42404,22.6370,Strong Up
2024-12-27 00:00:00-05:00,254.97,236.2430,0.41748,18.7270,Strong Up


In [12]:
# Visualize trends
fig = make_subplots(rows=3, cols=1, 
                    subplot_titles=['Price with Trend Colors', 'MA Slope', 'Price vs MA'],
                    vertical_spacing=0.05)

# Color map for trends
color_map = {
    'Strong Up': 'darkgreen',
    'Weak Up': 'lightgreen', 
    'Sideways': 'gray',
    'Weak Down': 'orange',
    'Strong Down': 'red'
}

# Group by trend and plot
for trend in data['Trend'].unique():
    if pd.notna(trend):
        trend_data = data[data['Trend'] == trend]
        if len(trend_data) > 0:
            fig.add_trace(go.Scatter(x=trend_data.index, y=trend_data['Close'],
                                   mode='markers', name=trend,
                                   marker=dict(color=color_map.get(trend, 'gray'), size=3)),
                         row=1, col=1)

# Add MA
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'],
                        name='SMA 50', line=dict(color='blue', width=2)),
             row=1, col=1)

# MA Slope
fig.add_trace(go.Scatter(x=data.index, y=data['MA_Slope'],
                        name='MA Slope', line=dict(color='purple')),
             row=2, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

# Price vs MA
fig.add_trace(go.Scatter(x=data.index, y=data['Price_vs_MA'],
                        name='Price - MA', line=dict(color='orange')),
             row=3, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)

fig.update_layout(height=900, title_text="Trend Analysis")
fig.show()

## 7. Moving Average Trading Strategy

Let's implement a simple moving average crossover strategy:
- **Buy Signal**: When fast MA crosses above slow MA
- **Sell Signal**: When fast MA crosses below slow MA

In [13]:
def ma_crossover_strategy(data, fast_period=20, slow_period=50):
    """
    Implement moving average crossover strategy
    
    Parameters:
    data (pd.DataFrame): Price data
    fast_period (int): Fast MA period
    slow_period (int): Slow MA period
    
    Returns:
    pd.DataFrame: DataFrame with strategy signals
    """
    df = data.copy()
    
    # Calculate MAs if not exist
    fast_ma = f'MA_{fast_period}'
    slow_ma = f'MA_{slow_period}'
    
    df[fast_ma] = simple_moving_average(df['Close'], fast_period)
    df[slow_ma] = simple_moving_average(df['Close'], slow_period)
    
    # Generate signals
    df['Signal'] = 0
    df.loc[df[fast_ma] > df[slow_ma], 'Signal'] = 1  # Buy
    df.loc[df[fast_ma] < df[slow_ma], 'Signal'] = -1  # Sell
    
    # Detect signal changes
    df['Signal_Change'] = df['Signal'].diff()
    df['Entry'] = (df['Signal_Change'] != 0) & (df['Signal_Change'].notna())
    
    # Mark specific entry types
    df['Buy_Entry'] = (df['Signal_Change'] == 2).astype(int)  # -1 to 1
    df['Sell_Entry'] = (df['Signal_Change'] == -2).astype(int)  # 1 to -1
    
    return df

# Apply strategy
strategy_data = ma_crossover_strategy(data, 20, 50)

# Find entry points
buy_entries = strategy_data[strategy_data['Buy_Entry'] == 1]
sell_entries = strategy_data[strategy_data['Sell_Entry'] == 1]

print(f"Strategy: {len(buy_entries)} buy signals, {len(sell_entries)} sell signals")
print("\nFirst 5 buy entries:")
print(buy_entries[['Close', 'MA_20', 'MA_50', 'Signal']].head())

Strategy: 14 buy signals, 13 sell signals

First 5 buy entries:
                            Close     MA_20     MA_50  Signal
Date                                                         
2020-04-30 00:00:00-04:00   71.21   66.3980   65.6948       1
2020-11-05 00:00:00-05:00  115.91  113.1945  113.1730       1
2021-04-19 00:00:00-04:00  131.73  124.0175  123.6694       1
2021-06-28 00:00:00-04:00  131.90  126.5260  126.3140       1
2021-11-04 00:00:00-04:00  147.95  144.5665  144.5386       1


In [14]:
# Visualize strategy
fig = go.Figure()

# Add price
fig.add_trace(go.Scatter(x=strategy_data.index, y=strategy_data['Close'],
                        name='Close Price', line=dict(color='black')))

# Add MAs
fig.add_trace(go.Scatter(x=strategy_data.index, y=strategy_data['MA_20'],
                        name='MA 20 (Fast)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=strategy_data.index, y=strategy_data['MA_50'],
                        name='MA 50 (Slow)', line=dict(color='red')))

# Add entry signals
fig.add_trace(go.Scatter(x=buy_entries.index, y=buy_entries['Close'],
                        mode='markers', name='Buy Signal',
                        marker=dict(color='green', size=10, symbol='triangle-up')))

fig.add_trace(go.Scatter(x=sell_entries.index, y=sell_entries['Close'],
                        mode='markers', name='Sell Signal',
                        marker=dict(color='red', size=10, symbol='triangle-down')))

fig.update_layout(
    title='Moving Average Crossover Strategy',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    height=600
)

fig.show()

## 8. Strategy Backtesting

Let's backtest our moving average strategy to evaluate its performance.

In [15]:
def backtest_strategy(data, initial_capital=10000):
    """
    Backtest the moving average crossover strategy
    
    Parameters:
    data (pd.DataFrame): Strategy data with signals
    initial_capital (float): Starting capital
    
    Returns:
    pd.DataFrame: DataFrame with portfolio performance
    """
    df = data.copy()
    
    # Initialize portfolio
    df['Position'] = df['Signal'].shift(1)  # Use previous day's signal
    df['Position'].fillna(0, inplace=True)
    
    # Calculate returns
    df['Returns'] = df['Close'].pct_change()
    df['Strategy_Returns'] = df['Position'] * df['Returns']
    
    # Calculate cumulative returns
    df['Cumulative_Returns'] = (1 + df['Returns']).cumprod()
    df['Cumulative_Strategy_Returns'] = (1 + df['Strategy_Returns']).cumprod()
    
    # Portfolio value
    df['Portfolio_Value'] = initial_capital * df['Cumulative_Strategy_Returns']
    df['Buy_Hold_Value'] = initial_capital * df['Cumulative_Returns']
    
    return df

# Run backtest
backtest_results = backtest_strategy(strategy_data)

# Calculate performance metrics
total_return_strategy = (backtest_results['Portfolio_Value'].iloc[-1] / 10000 - 1) * 100
total_return_buyhold = (backtest_results['Buy_Hold_Value'].iloc[-1] / 10000 - 1) * 100

# Annualized returns (assuming ~252 trading days per year)
years = len(backtest_results) / 252
annual_return_strategy = ((backtest_results['Portfolio_Value'].iloc[-1] / 10000) ** (1/years) - 1) * 100
annual_return_buyhold = ((backtest_results['Buy_Hold_Value'].iloc[-1] / 10000) ** (1/years) - 1) * 100

# Volatility (annualized)
strategy_vol = backtest_results['Strategy_Returns'].std() * np.sqrt(252) * 100
buyhold_vol = backtest_results['Returns'].std() * np.sqrt(252) * 100

# Sharpe ratio (assuming 0% risk-free rate)
sharpe_strategy = annual_return_strategy / strategy_vol if strategy_vol > 0 else 0
sharpe_buyhold = annual_return_buyhold / buyhold_vol if buyhold_vol > 0 else 0

print("BACKTEST RESULTS")
print("=" * 50)
print(f"Period: {backtest_results.index[0].date()} to {backtest_results.index[-1].date()}")
print(f"Duration: {years:.1f} years")
print("\nTOTAL RETURNS:")
print(f"Strategy: {total_return_strategy:.2f}%")
print(f"Buy & Hold: {total_return_buyhold:.2f}%")
print("\nANNUALIZED RETURNS:")
print(f"Strategy: {annual_return_strategy:.2f}%")
print(f"Buy & Hold: {annual_return_buyhold:.2f}%")
print("\nVOLATILITY (Annualized):")
print(f"Strategy: {strategy_vol:.2f}%")
print(f"Buy & Hold: {buyhold_vol:.2f}%")
print("\nSHARPE RATIO:")
print(f"Strategy: {sharpe_strategy:.2f}")
print(f"Buy & Hold: {sharpe_buyhold:.2f}")
print("\nFINAL VALUES:")
print(f"Strategy Portfolio: ${backtest_results['Portfolio_Value'].iloc[-1]:,.2f}")
print(f"Buy & Hold Portfolio: ${backtest_results['Buy_Hold_Value'].iloc[-1]:,.2f}")

BACKTEST RESULTS
Period: 2020-01-02 to 2024-12-30
Duration: 5.0 years

TOTAL RETURNS:
Strategy: -7.55%
Buy & Hold: 246.45%

ANNUALIZED RETURNS:
Strategy: -1.56%
Buy & Hold: 28.29%

VOLATILITY (Annualized):
Strategy: 29.55%
Buy & Hold: 31.69%

SHARPE RATIO:
Strategy: -0.05
Buy & Hold: 0.89

FINAL VALUES:
Strategy Portfolio: $9,245.42
Buy & Hold Portfolio: $34,644.73


In [16]:
# Visualize backtest results
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=['Portfolio Value Comparison', 'Daily Returns'],
                    vertical_spacing=0.1)

# Portfolio values
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Portfolio_Value'],
                        name='MA Strategy', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Buy_Hold_Value'],
                        name='Buy & Hold', line=dict(color='red')), row=1, col=1)

# Daily returns
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Strategy_Returns'] * 100,
                        name='Strategy Returns', line=dict(color='blue', width=1)), row=2, col=1)
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Returns'] * 100,
                        name='Buy & Hold Returns', line=dict(color='red', width=1)), row=2, col=1)

fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.update_layout(height=800, title_text="Strategy Performance Analysis")
fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
fig.update_yaxes(title_text="Daily Return (%)", row=2, col=1)

fig.show()

## 9. Advanced Analysis: Drawdown and Risk Metrics

Understanding risk is crucial in trading strategy evaluation. Beyond returns, we need to assess how much risk we're taking to achieve those returns.

### Key Risk Metrics:

#### 1. **Drawdown Analysis**
Drawdown measures the decline from a historical peak in portfolio value. It's one of the most important risk metrics.

**Formula**: `Drawdown = (Portfolio Value - Peak Value) / Peak Value`

- **Maximum Drawdown (MDD)**: The largest peak-to-trough decline
- **Duration**: How long it takes to recover from a drawdown
- **Frequency**: How often drawdowns occur

#### 2. **Win Rate**
The percentage of profitable trades or periods.

**Formula**: `Win Rate = (Number of Winning Periods / Total Trading Periods) × 100`

#### 3. **Risk-Adjusted Returns**
- **Sharpe Ratio**: Return per unit of risk (volatility)
- **Sortino Ratio**: Similar to Sharpe but only considers downside deviation
- **Calmar Ratio**: Annual return divided by maximum drawdown

#### 4. **Volatility Metrics**
- **Standard Deviation**: Measures the dispersion of returns
- **Downside Deviation**: Only considers negative returns
- **Beta**: Correlation with market movements

### Why These Metrics Matter:

1. **Risk Assessment**: High returns with high drawdowns may not be sustainable
2. **Psychological Impact**: Large drawdowns can lead to poor decision-making
3. **Capital Preservation**: Understanding worst-case scenarios helps with position sizing
4. **Strategy Comparison**: Risk-adjusted metrics allow fair comparison between strategies
5. **Regulatory Requirements**: Many funds have drawdown limits

### Interpreting Results:

- **Good Strategy**: Low maximum drawdown (<10%), high win rate (>50%), high Sharpe ratio (>1.0)
- **Risky Strategy**: High maximum drawdown (>20%), low win rate (<40%), volatile returns
- **Conservative Strategy**: Very low drawdowns but potentially lower returns

Let's calculate and visualize these important risk metrics:

In [17]:
def calculate_drawdown(portfolio_values):
    """
    Calculate drawdown series
    
    Parameters:
    portfolio_values (pd.Series): Portfolio value series
    
    Returns:
    pd.Series: Drawdown series
    """
    peak = portfolio_values.expanding(min_periods=1).max()
    drawdown = (portfolio_values - peak) / peak
    return drawdown

# Calculate drawdowns
backtest_results['Strategy_Drawdown'] = calculate_drawdown(backtest_results['Portfolio_Value'])
backtest_results['BuyHold_Drawdown'] = calculate_drawdown(backtest_results['Buy_Hold_Value'])

# Maximum drawdown
max_dd_strategy = backtest_results['Strategy_Drawdown'].min() * 100
max_dd_buyhold = backtest_results['BuyHold_Drawdown'].min() * 100

# Win rate for strategy
winning_trades = (backtest_results['Strategy_Returns'] > 0).sum()
total_trades = (backtest_results['Strategy_Returns'] != 0).sum()
win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0

print("RISK METRICS")
print("=" * 30)
print(f"Maximum Drawdown:")
print(f"  Strategy: {max_dd_strategy:.2f}%")
print(f"  Buy & Hold: {max_dd_buyhold:.2f}%")
print(f"\nWin Rate (Strategy): {win_rate:.1f}%")
print(f"Total Trading Days: {total_trades}")

# Visualize drawdowns
fig = go.Figure()

fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Strategy_Drawdown'] * 100,
                        name='Strategy Drawdown', line=dict(color='blue'), fill='tonexty'))
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['BuyHold_Drawdown'] * 100,
                        name='Buy & Hold Drawdown', line=dict(color='red'), fill='tonexty'))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Drawdown Analysis',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    height=500
)

fig.show()

RISK METRICS
Maximum Drawdown:
  Strategy: -56.88%
  Buy & Hold: -31.42%

Win Rate (Strategy): 50.6%
Total Trading Days: 1204


### Risk Metrics Analysis and Interpretation

Let's dive deeper into what these risk metrics tell us about our trading strategy:

#### **Understanding Drawdown Patterns:**

1. **Drawdown Magnitude**: How deep do the losses go?
   - < 5%: Very conservative, low risk
   - 5-15%: Moderate risk, acceptable for most investors
   - 15-30%: High risk, requires strong risk tolerance
   - > 30%: Extreme risk, may lead to emotional decisions

2. **Drawdown Duration**: How long do recoveries take?
   - Short recovery periods indicate resilient strategies
   - Long recovery periods may test investor patience
   - Multiple consecutive drawdowns are concerning

3. **Drawdown Frequency**: How often do losses occur?
   - Frequent small drawdowns may be preferable to rare large ones
   - Helps set realistic expectations for investors

#### **Win Rate Considerations:**

- **High Win Rate (>60%)**: Often indicates trend-following strategies
- **Medium Win Rate (40-60%)**: Balanced approach, typical for many strategies
- **Low Win Rate (<40%)**: May rely on a few large winners (momentum strategies)

**Important**: Win rate alone doesn't determine profitability. A strategy with 30% win rate but large average wins can outperform a 70% win rate strategy with small average wins.

#### **Risk-Adjusted Performance:**

The **Sharpe Ratio** helps us understand return per unit of risk:
- **> 2.0**: Excellent risk-adjusted returns
- **1.0 - 2.0**: Good risk-adjusted returns
- **0.5 - 1.0**: Adequate risk-adjusted returns
- **< 0.5**: Poor risk-adjusted returns

#### **Practical Implications:**

1. **Position Sizing**: Use maximum drawdown to determine appropriate position sizes
2. **Diversification**: High drawdowns suggest need for strategy diversification
3. **Market Timing**: Some strategies perform better in specific market conditions
4. **Psychological Preparedness**: Knowing worst-case scenarios helps maintain discipline

In [ ]:
# Additional Risk Analysis: Rolling Metrics
import matplotlib.dates as mdates

# Calculate rolling Sharpe ratio (252-day window)
rolling_window = 252
backtest_results['Rolling_Sharpe'] = (
    backtest_results['Strategy_Returns'].rolling(window=rolling_window).mean() * 252 / 
    (backtest_results['Strategy_Returns'].rolling(window=rolling_window).std() * np.sqrt(252))
)

# Calculate rolling maximum drawdown
def rolling_max_drawdown(series, window):
    """Calculate rolling maximum drawdown"""
    rolling_max = series.rolling(window=window, min_periods=1).max()
    rolling_dd = (series - rolling_max) / rolling_max
    return rolling_dd.rolling(window=window, min_periods=1).min()

backtest_results['Rolling_Max_DD'] = rolling_max_drawdown(
    backtest_results['Portfolio_Value'], rolling_window
) * 100

# Visualize rolling risk metrics
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=['Portfolio Value', 'Rolling Sharpe Ratio (1Y)', 'Rolling Max Drawdown (1Y)'],
                    vertical_spacing=0.08)

# Portfolio value
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Portfolio_Value'],
                        name='Strategy Portfolio', line=dict(color='blue')), row=1, col=1)

# Rolling Sharpe ratio
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Rolling_Sharpe'],
                        name='Rolling Sharpe', line=dict(color='green')), row=2, col=1)
fig.add_hline(y=1.0, line_dash="dash", line_color="red", 
              annotation_text="Good Threshold", row=2, col=1)

# Rolling maximum drawdown
fig.add_trace(go.Scatter(x=backtest_results.index, y=backtest_results['Rolling_Max_DD'],
                        name='Rolling Max DD', line=dict(color='red'), fill='tonexty'), row=3, col=1)
fig.add_hline(y=-15, line_dash="dash", line_color="orange", 
              annotation_text="High Risk Threshold", row=3, col=1)

fig.update_layout(height=900, title_text="Rolling Risk Metrics Analysis")
fig.update_yaxes(title_text="Value ($)", row=1, col=1)
fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=3, col=1)

fig.show()

# Risk-Return Scatter Analysis
print("\n" + "="*60)
print("RISK-RETURN ANALYSIS")
print("="*60)

# Calculate monthly returns for better granularity
monthly_returns = backtest_results['Strategy_Returns'].resample('M').apply(lambda x: (1 + x).prod() - 1)
monthly_volatility = monthly_returns.std() * np.sqrt(12) * 100
monthly_return = monthly_returns.mean() * 12 * 100

print(f"Monthly Analysis:")
print(f"  Average Monthly Return: {monthly_returns.mean()*100:.2f}%")
print(f"  Monthly Volatility: {monthly_returns.std()*100:.2f}%")
print(f"  Annualized Return: {monthly_return:.2f}%")
print(f"  Annualized Volatility: {monthly_volatility:.2f}%")
print(f"  Best Month: {monthly_returns.max()*100:.2f}%")
print(f"  Worst Month: {monthly_returns.min()*100:.2f}%")

# Value at Risk (VaR) - 95% confidence level
var_95 = np.percentile(backtest_results['Strategy_Returns'], 5) * 100
var_99 = np.percentile(backtest_results['Strategy_Returns'], 1) * 100

print(f"\nValue at Risk (VaR):")
print(f"  95% VaR: {var_95:.2f}% (daily)")
print(f"  99% VaR: {var_99:.2f}% (daily)")
print(f"  Interpretation: 95% of days, losses won't exceed {abs(var_95):.2f}%")

# Underwater plot (drawdown duration)
drawdown_data = backtest_results['Strategy_Drawdown'] * 100
print(f"\nDrawdown Statistics:")
print(f"  Current Drawdown: {drawdown_data.iloc[-1]:.2f}%")
print(f"  Average Drawdown: {drawdown_data[drawdown_data < 0].mean():.2f}%")
print(f"  Days in Drawdown: {len(drawdown_data[drawdown_data < 0])} / {len(drawdown_data)} ({len(drawdown_data[drawdown_data < 0])/len(drawdown_data)*100:.1f}%)")

# Recovery analysis
drawdown_periods = []
current_dd_start = None
for i, dd in enumerate(drawdown_data):
    if dd < -0.01 and current_dd_start is None:  # Start of drawdown
        current_dd_start = i
    elif dd >= -0.01 and current_dd_start is not None:  # End of drawdown
        recovery_days = i - current_dd_start
        max_dd_in_period = drawdown_data[current_dd_start:i].min()
        drawdown_periods.append({
            'start': current_dd_start,
            'end': i,
            'duration': recovery_days,
            'max_drawdown': max_dd_in_period
        })
        current_dd_start = None

if drawdown_periods:
    avg_recovery = np.mean([dd['duration'] for dd in drawdown_periods])
    max_recovery = max([dd['duration'] for dd in drawdown_periods])
    print(f"  Average Recovery Time: {avg_recovery:.0f} days")
    print(f"  Longest Recovery Time: {max_recovery} days")
    print(f"  Number of Drawdown Periods: {len(drawdown_periods)}")
else:
    print("  No significant drawdown periods found")

### Risk Management Best Practices

Based on our risk analysis, here are key takeaways for practical implementation:

#### **1. Position Sizing Guidelines:**
```
Position Size = Account Risk / Strategy Risk
Where:
- Account Risk = % of capital you're willing to lose (typically 1-2% per trade)
- Strategy Risk = Expected loss based on historical drawdowns
```

#### **2. Drawdown Management:**
- **Stop Trading Rule**: Stop strategy if drawdown exceeds 2x historical maximum
- **Reduce Position Size**: Cut positions by 50% if drawdown reaches 75% of historical max
- **Diversification**: Never allocate more than 25% of capital to single strategy

#### **3. Performance Monitoring:**
- **Daily**: Monitor current drawdown and position sizes
- **Weekly**: Review rolling Sharpe ratio and recent performance
- **Monthly**: Comprehensive risk metrics analysis
- **Quarterly**: Strategy review and parameter optimization

#### **4. Warning Signs to Watch:**
🚨 **Red Flags:**
- Drawdown exceeding historical maximum by 50%
- Rolling Sharpe ratio below 0.5 for 6+ months
- Win rate declining significantly from historical average
- Recovery time increasing beyond normal ranges

✅ **Good Signs:**
- Consistent risk-adjusted returns
- Drawdowns within expected ranges
- Quick recovery from losses
- Stable or improving rolling metrics

#### **5. Risk-Adjusted Position Sizing Example:**
```python
# Example calculation
max_drawdown = 15%  # From our analysis
account_risk_per_trade = 1%  # Conservative approach
position_multiplier = account_risk_per_trade / max_drawdown
# = 1% / 15% = 0.067 (or 6.7% of intended position size)
```

This means if your strategy historically had 15% maximum drawdown, you should only risk 6.7% of your normal position size to limit account risk to 1% per trade.

#### **6. Psychological Considerations:**
- **Expect Drawdowns**: They are normal and inevitable
- **Stay Disciplined**: Don't abandon strategy during temporary losses
- **Plan Ahead**: Know your exit criteria before entering trades
- **Keep Records**: Document decisions for future learning

Remember: **The goal is not to eliminate risk, but to understand and manage it appropriately for your risk tolerance and investment objectives.**

## 10. Exercise: Optimize MA Strategy

Let's test different MA periods to find the optimal combination.

In [ ]:
def optimize_ma_strategy(data, fast_range, slow_range):
    """
    Optimize moving average strategy parameters
    
    Parameters:
    data (pd.DataFrame): Price data
    fast_range (list): Range of fast MA periods to test
    slow_range (list): Range of slow MA periods to test
    
    Returns:
    pd.DataFrame: Optimization results
    """
    results = []
    
    for fast in fast_range:
        for slow in slow_range:
            if fast >= slow:  # Skip invalid combinations
                continue
                
            # Test strategy
            strategy_data = ma_crossover_strategy(data, fast, slow)
            backtest_data = backtest_strategy(strategy_data)
            
            # Calculate metrics
            total_return = (backtest_data['Portfolio_Value'].iloc[-1] / 10000 - 1) * 100
            volatility = backtest_data['Strategy_Returns'].std() * np.sqrt(252) * 100
            sharpe = (total_return / len(backtest_data) * 252) / volatility if volatility > 0 else 0
            
            drawdown = calculate_drawdown(backtest_data['Portfolio_Value'])
            max_drawdown = drawdown.min() * 100
            
            results.append({
                'Fast_MA': fast,
                'Slow_MA': slow,
                'Total_Return': total_return,
                'Volatility': volatility,
                'Sharpe_Ratio': sharpe,
                'Max_Drawdown': max_drawdown
            })
    
    return pd.DataFrame(results)

# Run optimization (limited range for demo)
print("Running strategy optimization...")
fast_periods = [10, 15, 20, 25]
slow_periods = [30, 40, 50, 60]

optimization_results = optimize_ma_strategy(data, fast_periods, slow_periods)

# Sort by Sharpe ratio
optimization_results = optimization_results.sort_values('Sharpe_Ratio', ascending=False)

print("\nTop 10 Strategy Combinations (by Sharpe Ratio):")
print(optimization_results.head(10).round(2))

In [ ]:
# Visualize optimization results
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Total Return (%)', 'Sharpe Ratio', 
                                   'Volatility (%)', 'Max Drawdown (%)'])

# Create pivot tables for heatmaps
return_pivot = optimization_results.pivot(index='Slow_MA', columns='Fast_MA', values='Total_Return')
sharpe_pivot = optimization_results.pivot(index='Slow_MA', columns='Fast_MA', values='Sharpe_Ratio')
vol_pivot = optimization_results.pivot(index='Slow_MA', columns='Fast_MA', values='Volatility')
dd_pivot = optimization_results.pivot(index='Slow_MA', columns='Fast_MA', values='Max_Drawdown')

# Add heatmaps
fig.add_trace(go.Heatmap(z=return_pivot.values, x=return_pivot.columns, y=return_pivot.index,
                        colorscale='RdYlGn', showscale=False), row=1, col=1)
fig.add_trace(go.Heatmap(z=sharpe_pivot.values, x=sharpe_pivot.columns, y=sharpe_pivot.index,
                        colorscale='RdYlGn', showscale=False), row=1, col=2)
fig.add_trace(go.Heatmap(z=vol_pivot.values, x=vol_pivot.columns, y=vol_pivot.index,
                        colorscale='RdYlGn_r', showscale=False), row=2, col=1)
fig.add_trace(go.Heatmap(z=dd_pivot.values, x=dd_pivot.columns, y=dd_pivot.index,
                        colorscale='RdYlGn_r', showscale=False), row=2, col=2)

fig.update_layout(height=700, title_text="Strategy Optimization Heatmaps")
fig.show()

# Best combination
best_strategy = optimization_results.iloc[0]
print(f"\nBest Strategy Combination:")
print(f"Fast MA: {best_strategy['Fast_MA']} days")
print(f"Slow MA: {best_strategy['Slow_MA']} days")
print(f"Total Return: {best_strategy['Total_Return']:.2f}%")
print(f"Sharpe Ratio: {best_strategy['Sharpe_Ratio']:.2f}")
print(f"Max Drawdown: {best_strategy['Max_Drawdown']:.2f}%")

## 11. Key Takeaways

### Moving Averages Summary:

1. **Simple Moving Average (SMA)**:
   - Equal weight to all periods
   - Smooth but lagging indicator
   - Good for trend identification

2. **Exponential Moving Average (EMA)**:
   - More weight to recent prices
   - More responsive to price changes
   - Better for signal generation

3. **MACD Indicator**:
   - Combines trend and momentum
   - Signal line crossovers for entries
   - Histogram shows momentum changes

4. **Strategy Performance**:
   - Parameter optimization is crucial
   - Consider risk-adjusted returns (Sharpe ratio)
   - Monitor drawdowns and volatility

### Best Practices:
- Test multiple timeframes
- Use proper risk management
- Consider transaction costs
- Validate on out-of-sample data
- Combine with other indicators

## 12. Exercises for Further Practice

1. **Implement Triple Moving Average Strategy**: Use three MAs (short, medium, long) for signal generation

2. **Add Stop Loss and Take Profit**: Modify the strategy to include risk management rules

3. **Test Different Assets**: Apply the strategy to different stocks, forex, or crypto

4. **Volume Confirmation**: Only take signals when volume is above average

5. **Multi-timeframe Analysis**: Use daily signals with hourly execution

6. **Machine Learning Enhancement**: Use ML to predict optimal MA periods dynamically